In [1]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
import pickle, os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import sqlite3 as sql
from pathlib import Path

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [11]:
import torch
print(torch.cuda.is_available())  # should be True
print(torch.cuda.get_device_name(0))

True
Tesla V100-SXM2-32GB


In [5]:
import database as db
path = Path(os.path.abspath(""))
db_path = path / "games.db"
conn = sql.connect(db_path)
cur = conn.cursor()
data_path = path / ".." / "data"

data = pd.read_sql(db.get_text_gamedata(as_text=True, cur=cur), conn, index_col="id_game")
docs = pd.Series([". ".join(data.loc[i]) for i in data.index], index=data.index)
print(docs.head())

<>:14: SyntaxWarning: invalid escape sequence '\w'
<>:14: SyntaxWarning: invalid escape sequence '\w'
/run/nvme/job_31979174/tmp/ipykernel_4006225/2866284385.py:14: SyntaxWarning: invalid escape sequence '\w'
  token_pattern=r"\b\w+\b",


id_game
25097    The Legend of Zelda: Ocarina of Time. Adventur...
54751    Soulcalibur. Action, Fighting. Soulcalibur (ソウ...
22511    The Legend of Zelda: Breath of the Wild. Adven...
27024    Super Mario Galaxy. Platformer. Become Mario a...
27036    Super Mario Galaxy 2. Platformer. Mario is inv...
dtype: object


'####### TF-IDF VECTORS ###########\nvectorizer = TfidfVectorizer(\n        lowercase=True,\n        token_pattern=r"\x08\\w+\x08",\n        dtype=np.float32,\n        min_df=5,\n        max_df=0.8,\n        ngram_range=(1, 1)  # using ngrams will make vectors huuuuuge\n    )\nX = vectorizer.fit_transform(docs)\ntfidf_matrix = pd.DataFrame.sparse.from_spmatrix(\n                    X, \n                    index=docs.index, \n                    columns=vectorizer.get_feature_names_out()\n                    )\npickle.dump(tfidf_matrix, open(data_path / "tf_idf", "wb"))\n\nprint("tf-idf vectors done")'

In [12]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", device="cuda")
model.encode("meaning text")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/526 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

array([-2.70463135e-02, -1.87658533e-01,  6.88524693e-02,  2.50681102e-01,
       -2.38374062e-02, -2.68407464e-01,  3.47917557e-01,  2.14069143e-01,
        5.29131182e-02,  1.43346548e-01, -3.38119045e-02, -9.19044614e-02,
        3.09854954e-01, -1.92895114e-01,  1.75511777e-01, -1.12418011e-01,
        1.84954256e-01, -8.63439217e-02, -1.48092598e-01, -1.48829579e-01,
        5.88362575e-01,  2.76139855e-01,  4.11153615e-01, -2.45536983e-01,
        6.91382065e-02,  2.61250556e-01, -2.90720016e-01, -2.92280644e-01,
        1.36718480e-02, -6.22682348e-02, -1.53779030e-01,  3.51010680e-01,
        5.64502418e-01,  4.67832267e-01,  1.02260254e-01,  5.00614226e-01,
        1.85492545e-01, -1.33941304e-02,  2.39082590e-01,  1.74850062e-01,
       -4.18368787e-01,  1.64771572e-01, -2.87071884e-01,  4.85542327e-01,
        2.73168892e-01, -2.08208375e-02,  1.43581498e-02, -5.79878315e-03,
        7.57449791e-02,  3.38633746e-01,  3.46494839e-02, -1.01947114e-02,
       -3.23141247e-01,  

In [16]:
with torch.no_grad():   # important for indexing
    embeddings = model.encode(
        docs.astype(str).tolist(),  # Series → list
        batch_size=128,             # tune this
        show_progress_bar=True,
        convert_to_numpy=True,
        normalize_embeddings=True   # VERY important for search
    )
#model.encode(docs.values)

Batches:   0%|          | 0/73 [00:00<?, ?it/s]

In [19]:
embeddings[:2].shape

(2, 384)

In [21]:
embedddings_df = pd.DataFrame(embeddings, index=docs.index)
embedddings_df.head()

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
id_game,,,,,,,,,,,,,,,,,,,,,
25097,-0.093967,-0.027472,-0.000640,0.063278,-0.035477,-0.054920,0.093295,-0.018577,0.025598,-0.008290,...,0.037465,-0.029124,0.041367,-0.006736,-0.037276,0.002694,0.017802,0.009456,0.002117,0.023315
54751,-0.055039,0.009732,-0.090044,-0.025392,-0.044478,0.062568,0.026969,-0.067745,0.034086,0.097463,...,0.036372,0.092768,-0.092867,0.045560,-0.070361,0.039498,-0.017421,-0.061471,-0.033166,-0.027482
22511,-0.077263,0.009459,-0.031007,0.070095,0.008252,-0.067667,0.101530,-0.052462,0.043279,-0.028803,...,-0.008765,0.082399,-0.012495,0.050213,-0.099979,-0.040892,-0.006631,-0.019071,-0.024008,0.020125
27024,-0.056671,0.013248,-0.008285,0.016889,-0.006815,-0.111607,0.020833,-0.014201,-0.072550,0.007345,...,0.063899,0.019331,-0.016313,-0.016004,0.004065,0.111765,-0.028148,-0.008075,-0.046732,-0.012461
27036,-0.026780,0.046708,0.015584,0.056661,0.049057,-0.099242,0.059943,-0.003636,-0.025016,-0.013239,...,0.051173,-0.010225,-0.007532,-0.034270,0.020828,0.080152,0.009499,-0.005443,-0.079731,0.010954


In [33]:
from sentence_transformers.util import cos_sim

score = cos_sim(
    embedddings_df.loc[2551],
    embedddings_df.loc[28]
)

score

tensor([[0.3182]])

In [34]:
pickle.dump(embedddings_df, open("neural_embeddings", "wb"))

In [35]:
!pip install faiss

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [53]:
import faiss

dim = 384  # your MiniLM embedding size
index = faiss.IndexFlatIP(dim)

In [54]:
emb_matrix = np.vstack(embedddings_df.values).astype('float32')
index.add(emb_matrix)
# mapping to gameIDs
faiss_id_to_db_id = embedddings_df.index.to_numpy()

In [68]:
import copy
query = copy.deepcopy(embedddings_df.loc[28].values)
D, I = index.search(query.reshape(1, -1), 6)
top_ids = list(faiss_id_to_db_id[I[0]])
top_ids.remove(28)
top_ids

[4544, 391395, 4546, 19401, 530497]

In [69]:
cur.execute(f"""
SELECT name, id_game FROM game
WHERE id_game IN {tuple(map(str, top_ids))}
""").fetchall()

[('Red Dead Redemption', 4544),
 ('Red Dead Redemption: Undead Nightmare', 4546),
 ('Helldorado', 19401),
 ('West of Dead', 391395),
 ('Evil West', 530497)]